# Mục đích

- Đảm bảo chắc chắn là class chỉ có 1 đối tượng (instance) duy nhất, và cung cấp một điểm truy cập global đến đối tượng đó

# Sử dụng

- Khi mà app chỉ cần duy nhất một đối tượng.
- Yêu cầu:
    - Được khởi tạo lười (lazy initialization): phương pháp trì hoãn việc khởi tạo (hoặc nạp) dữ liệu cho đến khi chúng thực sự cần đến.
    - Có thể truy cập global
    
# Implement

## Cách 1: [tham khảo](https://sourcemaking.com/design_patterns/singleton/python/1), [Singletons and metaclasses](https://learning.oreilly.com/library/view/learning-python-design/9781785888038/ch02s05.html)

Trước tiên phải hiểu `metaclass`, đó là class của class, nghĩa là class sẽ là 1 thực thể của `metaclass`

`metaclass` cho phép ta tạo ra kiểu class của riêng mình, từ 1 class đã định nghĩa trước.

Chi tiết đây này, trong Python mọi thứ đều là object, ví dụ nếu `a=5` thì `type(a)` trả về `<type 'int'>`. Nghĩa là a kiểu int. Tuy nhiên `type(int)` sẽ trả về `<type 'type'>`. Nó gợi ý về sự hiện diện của `metaclass`, vì int là một lớp kiểu `type`

Định nghĩa của 1 class được quyết định bởi `metaclass`, vì vậy khi tạo 1 class, ví dụ `class A`, Python tạo nó bằng cách: `A = type(name, base, dict)`, trong đó:
- name: tên của class
- base: là base class
- dict: là biến các thuộc tính 

Bây giờ, nếu một class có một `metaclass` được định nghĩa trước, ví dụ đặt tên là `MetaKls`, lúc này Python sẽ tạo class `A =  MetaKls(name, bases, dict)`

Xem ví dụ sau về `metaclass`

In [4]:
class MyInt(type):
    def __call__(cls, *args, **kwargs):
        print("***** Hello, here is My int *****")
        print("Now, do whatever you want with these objects...")
        return type.__call__(cls, *args, **kwargs)
    
class int(metaclass=MyInt):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "This is MyInt object"
        
i = int(1,2)
print(i)

***** Hello, here is My int *****
Now, do whatever you want with these objects...
This is MyInt object


Method `__call__` được gọi khi một đội tượng được tạo từ một class đã tồn tại. Trong đoạn code trên, khi chúng ta tạo 1 thực thể của class int thì method `__call__` của `MetaKls` được gọi, chứng tỏ `metaclass` điểu khiển quá trình khởi tạo thực thể, điều này quá toẹt vời, hế hế...

Chúng ta có thê sử dụng tính chất trên để tạo ra `Singleton` pattern

**Chú ý:** để điều khiển quá trình khởi động của một class, `metaclass` sẽ ghi đè phương thức `__new__` và `__init__`

Ta tạo một `metaclass` Singleton:

In [40]:
class SingletonMetaClass(type):
    
    def __init__(cls, name, bases, attrs, **kwargs):
        super().__init__(name, bases, attrs)
        cls.__instance = None
        print("__init__ of Singleton")
        
    def __call__(cls, *args, **kwargs):
        print("__call__ of Singleton")
        if cls.__instance is None:
            cls.__instance = super().__call__(*args, **kwargs)
        return cls.__instance

Bây giờ tạo 1 Singleton class từ thực thể trên

In [41]:
class MySingleton(metaclass=SingletonMetaClass):
    def __init__(self):
        print("__init__ of MySingleton")

__init__ of Singleton


Bây giờ tạo thực thể từ class trên xem nó có là 1 không nhá

In [42]:
m1 = MySingleton()
print(m1)
print("----------------")

m2 = MySingleton()
print(m2)


__call__ of Singleton
__init__ of MySingleton
----------------
__call__ of Singleton


Tèn tén, kết quả in ra và kiểm tra là giống nhau, vậy nó đảm bảo tính chất là luôn luôn chỉ có duy nhất 1 thực thể của class được tạo ra rồi nhá

Còn `lazy initialization` thì sao nhỉ, trong Python có thể dùng `@property` theo link sau:

[how-can-i-delay-the-init-call-until-an-attribute-is-accessed](https://stackoverflow.com/questions/45194553/how-can-i-delay-the-init-call-until-an-attribute-is-accessed)

[lazy-properties](https://stevenloria.com/lazy-properties/)

Ví dụ bên dưới, thuộc tính `_foo` chỉ được khởi tạo khi gọi nó, nếu không nó vẫn là `None`

In [45]:
class MySingleton1(metaclass=SingletonMetaClass):
    def __init__(self):
        print("__init__ of MySingleton1")
        self._foo = None

    @property
    def foo(self):
        if not self._foo:
            print("lazy initialization")
            self._foo =  [1,2,3]
        return self._foo
    
ms1 = MySingleton1()
print(ms1._foo)
print(ms1.foo)

ms2 = MySingleton1()
print(ms2._foo)

__init__ of Singleton
__call__ of Singleton
__init__ of MySingleton1
None
lazy initialization
[1, 2, 3]
__call__ of Singleton
[1, 2, 3]


## Cách 2: [tham khảo](https://www.tutorialspoint.com/python_design_patterns/python_design_patterns_singleton.htm)

Hoạt động của cách này:
- dùng 1 biến private để lưu thực thể của class đó, 
- dùng 1 static method tạo thực thể, xong lưu vào biến pravite
- thực thể chỉ có thể tạo bằng static method, hoặc tạo 1 lần duy nhất bằng cách gọi tên class, tạo lần 2 bằng tên class sẽ báo lỗi


In [3]:
class Singleton2:
    __instance = None
    
    @staticmethod
    def get_instance():
        if Singleton2.__instance is None:
            Singleton2()
        return Singleton2.__instance
    
    def __init__(self):
        if Singleton2.__instance is not None:
            raise Exception("This class is singleton!")
        else:
            Singleton2.__instance = self

Bây giờ, tạo thực thể của class trên để test nhá :D

In [4]:
s = Singleton2()
print(s)
s = Singleton2.get_instance()
print(s)
s = Singleton2.get_instance()
print(s)

Tèn tén, giống nhau rồi nhá, bây giờ tạo một thực thể mới từ tên class không qua static method xem thế nào nhá

In [5]:
s = Singleton2()

Exception: This class is singleton!

Há há, lỗi rồi...vậy là đạt yêu cầu

Tham khảo phần **Lazy initialization** ở cách 1


## Cách 3: [tham khảo branch forked](https://github.com/PhungXuanAnh/python-patterns/blob/master/patterns/creational/borg.py)

Cách này sử dụng một tính chất của Python, đó là các thuộc tính của 1 thực thể thì được lưu trong 1 dictionary tên là `__dict__`. Vậy là bây giờ đơn giản là làm cho tất cả các thực thể có cùng `__dict__`, tức là cùng thuộc tính thì sẽ đáp ứng yêu cầu có duy nhất 1 thực thể được tạo ra, phải không nhể 

Trong cách này:
- Biến `__shared_state` sẽ dùng để lưu thuộc tính share giữa các thực thể
- Khi tạo thực thể mới thì gán `__shared_state` cho `__dict__`


# Áp dụng trong tình huống thực tế

https://learning.oreilly.com/library/view/learning-python-design/9781785888038/ch02s06.html

https://learning.oreilly.com/library/view/learning-python-design/9781785888038/ch02s07.html
